# Lorenz system

The Lorenz system is described by a system of three ordinary differential equations,
$$
\begin{align}
\frac{dx}{dt}&=f(x,y,z)=a(y-x)\\
\frac{dy}{dt}&=g(x,y,z)=x(b-z)-y\\
\frac{dz}{dt}&=h(x,y,z)=xy-cz
\end{align},
$$
wherein $a,b,c\in\mathbb{R}$ are constant system parameters.

In [2]:
from plotly import offline as py
from plotly import graph_objs as go

from dynamics import lorenz_rk4

py.init_notebook_mode(connected=True)

a = 10
c = 8/3

## Lyapunov exponent

Subject of the present notebook is to find the largest Lyapunov exponent for the Lorenz system with $a=10,c=8/3$ for $b=28$ and $b=100$. As a remainder the Lyapunov exponent of a time series $x_0,\dots,x_n$ is defined as,
$$
\lambda=\lim_{n\to\infty}\frac{1}{n}\sum^{n-1}_{i=0}\ln\left|f^\prime(x_i)\right|.
$$
Becasue we discretize the continous Lorenz system we can directly use this definition for numerical calculations, prior to we need to extend the definition of the Lyapunov exponent for the multivariate case $\boldsymbol{x}_0,\dots,\boldsymbol{x}_n$:
$$
\lambda_i=\lim_{n\to\infty}\frac{1}{n}\sum^{n-1}_{i=0}\ln\left|\nabla f^\prime(\boldsymbol{x}_i)\right|,
$$
where $f$ corresponds to the evolution of the $i$th coordinate and $\lambda_i$ is the Lyapunov exponent in the $i$th dimension. The largest Lyapunov exponent is then to be taken accross all dimensions, thus,
$$\lambda=\max\{\lambda_1,\dots,\lambda_n\}.$$

Back to the Lorenz system which is three dimensional. We calculate the gradients of the evolution functions:
$$
\begin{align}
\nabla f(x,y,z)&=(-a, +a, 0)^T\\
\nabla g(x,y,z)&=(b, -1, -x)^T\\
\nabla h(x,y,z)&=(y, x, -c)^T
\end{align}
$$
In order to obtain a scalar we evaluate the gradient at the given coordinates, thus,
$$
\begin{align}
\lambda_x&=\frac{1}{2n}\sum^{n-1}_{i=0}\ln\left[2a^2\right]\\
\lambda_y&=\frac{1}{2n}\sum^{n-1}_{i=0}\ln\left[b^2+1+x_i^2\right]=C_y+\frac{1}{n}\sum^{n-1}_{i=0}\ln\left[x_i\right]\\
\lambda_z&=\frac{1}{2n}\sum^{n-1}_{i=0}\ln\left[y_i^2+x_i^2+c^2\right]=C_z+\frac{1}{2n}\sum^{n-1}_{i=0}\ln\left[x_i^2+y_i^2\right],
\end{align}
$$
we can already rule out $\lambda_x$ as being the largest Lyapunov exponent, as it corresponds to a constant value. We also discard constant values from $\lambda_y,\lambda_z$ as they should not change the dynamics.

In [14]:
def lyapunov_naive(x0, y0, z0, b, N, h):
    x, y, z = lorenz_rk4(x0, y0, z0, a, b, c, N, h)
    
    return np.max([
        np.log(np.abs(x)).mean(),
        np.log(x**2 + y**2).mean() / 2
    ])

In [19]:
x0, y0, z0 = np.random.normal(1, 3, size=9).reshape(3, 3)

[lyapunov_naive(x0[i], y0[i], z0[i], 28, 100000, 1e-3) for i in range(3)]

[2.041129184320763, 2.04098332187096, 2.063258528775639]

In [20]:
[lyapunov_naive(x0[i], y0[i], z0[i], 100, 100000, 1e-3) for i in range(3)]

[2.958060375995601, 2.9984851790563036, 2.976283606159797]

We conclude that the Lorenz system with $b=100$ has a greater Lyapunov exponent than the Lorenz system with $b=28$.